In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import cv2
import os

In [4]:
def detect_faces(image):
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    # detect face
    face_pos = face_classifier.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40))
    return face_pos

In [5]:
def process_image(image, x, y, w, h):
    # extract the face ROI, resize it, and convert it to grayscale
    face = image[y : y + h, x : x + w]
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    return face

In [ ]:
def load_face_dataset(folder_path, min_samples = 1):
    # get names from filename
    names = []
    for filename in os.listdir(folder_path):
        name = filename.split("_")[0]
        names.append(name)
    
    # store unique names only
    (names, name_counts) = np.unique(names, return_counts = True)
    names = names.tolist()
	
    # extract faces and associated labels
    faces = []
    labels = []
    for filename in os.listdir(folder_path):
        name = filename.split("_")[0]
        # check for minimum amount of samples
        if name_counts[names.index(name)] < min_samples:
            continue
        
        # load image
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        # detect faces
        face_pos = detect_faces(image)
        for (x, y, w, h) in face_pos:
            face = process_image(image, x, y, w, h)
            faces.append(face)
            labels.append(name)
    
    # convert lists to NumPy arrays
    faces = np.array(faces, dtype="object")
    labels = np.array(labels)
    # return a 2-tuple of the faces and labels
    return (faces, labels)

In [ ]:
def recognize_LBPH(trainX, trainY, testX, testY):
    # train recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create(radius=2, neighbors=16, grid_x=8, grid_y=8)
    recognizer.train(trainX, trainY)
    
    predictions = []
    confidence = []
    # loop over the test data
    for i in range(0, len(testX)):
        # classify the face and update the list of predictions and confidence scores
        (prediction, conf) = recognizer.predict(testX[i])
        predictions.append(prediction)
        confidence.append(conf)
        pred_name = le.inverse_transform([predictions[i]])[0]
        actual_name = le.classes_[testY[i]]
        # confidence of the prediction (i.e., chi-squared distance; the *lower* the distance is the *more confident* the prediction is)
        print("[INFO] prediction: {}, actual: {}, confidence: {:.2f}".format( pred_name, actual_name, confidence[i]))
    # show the classification report
    print("Classification report:\n", classification_report(testY, predictions, target_names=le.classes_))

In [ ]:
folder_path = "folder path"

# load face dataset
(faces, labels) = load_face_dataset(folder_path)
# encode the string labels as integers
le = LabelEncoder()
labels = le.fit_transform(labels)
# construct our training and testing split
(trainX, testX, trainY, testY) = train_test_split(faces, labels, test_size=0.25, stratify=labels, random_state=42)

# LBPH face recognition
recognize_LBPH(trainX, trainY, testX, testY)